In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

# Emission
Copied from part 3

In [2]:
def train_emission(filename):
    """
    Returns - a dictionary containing emission parameters
    """
    with open(filename, encoding="utf8") as f:
        lines = f.readlines()
    
    # for each state y, keep track of each observation count i.e. count (y -> x)
    # before eg: {state1: {obs1: 1, obs2: 5}, state2: {obs1: 4}}
    emission_dict = {}
    
    # update emission_dict for state with count(y -> x) = 0
    # after eg: {state1: {obs1: 1, obs2: 5}, state2: {obs1: 4, obs2: 0}}
    observations = set()
    
    for line in lines:
        split_line = line.split()
        
        # process only valid lines
        if len(split_line) == 2:
            obs, state = split_line[0], split_line[1]
            
            observations.add(obs)
            
            if state not in emission_dict:
                emission_dict[state] = {}
                
            if obs not in emission_dict[state]:
                emission_dict[state][obs] = 1
            else:
                emission_dict[state][obs] += 1

    for k, v in emission_dict.items():
        for obs in observations:
            if obs not in v:
                emission_dict[k][obs] = 0
    
    return emission_dict

In [3]:
def get_emission_params_fixed(emission_dict, state, obs, k=0.5):
    
    if state not in emission_dict:
        raise Exception("State not in emission dict")
    
    state_data = emission_dict[state]
    count_y = sum(state_data.values()) # count(y)
    
    if obs == "#UNK#":
        count_y_to_x = k
    else:
        count_y_to_x = state_data[obs] # count(y -> x)
    
    return count_y_to_x / (count_y + k)

# Transitions
Copied from part 3

In [4]:
def train_transition(filename):
    """
    Returns - a dictionary containing transition parameters
    """
    with open(filename, encoding="utf8") as f:
        lines = f.readlines()
    
    # for each state u, keep track of each state count i.e. count (u,v)
    # before eg: {START: {y1: 1, y2: 5}, y1: {y1: 3, y2: 4, STOP: 1}, y2: {y1: 1, STOP: 3}}
    transition_dict = {}
    
    # after eg: {START: {y1: 1, y2: 5, STOP: 0}, y1: {y1: 3, y2: 4, STOP: 1}, y2: {y1: 1, y2: 0, STOP: 3}}
    states = set()
    states.add('STOP')
    
    prev_state = 'START'
        
    for line in lines:
        split_line = line.split()
        
        if prev_state not in transition_dict:
            transition_dict[prev_state] = {}
                
        # can only be START or STOP
        if len(split_line) < 2:
            if 'STOP' not in transition_dict[prev_state]:
                transition_dict[prev_state]['STOP'] = 0
            
            transition_dict[prev_state]['STOP'] += 1
            prev_state = 'START'
        
        # processing the sentence
        elif len(split_line) == 2:
            curr_state = split_line[1]
            states.add(curr_state)
           
            if curr_state not in transition_dict[prev_state]:
                transition_dict[prev_state][curr_state] = 0
            
            transition_dict[prev_state][curr_state] += 1
            prev_state = curr_state
    
    for k, v in transition_dict.items():
        for state in states:
            if state not in v:
                transition_dict[k][state] = 0
    
    return transition_dict

In [5]:
def get_transition_params(transition_dict, u, v):
    
    if u not in transition_dict:
        raise Exception("State u not in transition dict")
        
    if v not in transition_dict[u]:
        raise Exception("State v not in transition dict")
    
    state_data = transition_dict[u]
    
    count_u_to_v = state_data[v] # count(u,v)
    count_u = sum(state_data.values()) # count(u)
            
    return count_u_to_v / count_u

# Helper Functions
Copied from part 3

In [6]:
def obtain_all_obs(emission_dict):
    """
    Obtain all distinct observations words in the emission_dict.
    Purpose: This helps us identify words in Test Set that do not exist in the Training Set (or the emission_dict)
    Returns - Set of Strings.
    """
    all_observations = set()
    
    for s_to_obs_dict in emission_dict.values():
        for obs in s_to_obs_dict.keys():
            all_observations.add(obs)
            
    return all_observations

def preprocess_sentence(sentence, training_set_words):
    """
    sentence - a list of Strings (words or observations)
    Returns - a list of Strings, where Strings not in training_set_words are replaced by "#UNK#"
    """
    return [ word if word in training_set_words else "#UNK#" for word in sentence ]

# Training

In [7]:
def train(filename):
    """
    Returns - A 2-tuple (Dict, Dict): emission and transition parameters
    """
    return train_emission(filename), train_transition(filename)

# Testing Grounds. Before writing the actual function.

In [8]:
emission_dict, transition_dict = train('../dataset/EN/train')
training_set_words = obtain_all_obs(emission_dict)
all_states = list(emission_dict.keys())

sentence = "The quick brown fox jumps ."
sentence = sentence.split(' ')
proc_sent = preprocess_sentence(sentence, training_set_words)
print(sentence)
print(proc_sent)

proc_sent = ["start"] + proc_sent + ["stop"]
print(proc_sent)

n = len(proc_sent)
k = 3

# Pi np 
P = np.zeros( (len(proc_sent), len(all_states), k) ) 
# Backtrace Table
B = [ [ ['-', '-', '-'] for x in all_states ] for y in proc_sent ]

['The', 'quick', 'brown', 'fox', 'jumps', '.']
['The', 'quick', '#UNK#', 'fox', 'jumps', '.']
['start', 'The', 'quick', '#UNK#', 'fox', 'jumps', '.', 'stop']


In [9]:
a = lambda u, v: get_transition_params(transition_dict, u, v)
b = lambda state, obs: get_emission_params_fixed(emission_dict, state, obs, k=0.5)

# Base Case at j=1
t = np.array([ a('START', v) for v in all_states ])
e = np.array([ b(v, proc_sent[1]) for v in all_states ])
P[1, :, 0] = t * e
P[1, :, 1:] = 0 # There can be no 2nd or 3rd best path when j=1.

B[1] = [ ["START", '-', '-'] for row in B[1] ]

In [10]:
# Recursive Forward Step for j=2 to n-2 (inclusive). Where n = len(proc_sent) => Layer n is the last layer (STOP).
# At layer n-1, we perform termination step, as layer n is STOP. 
# Recall that proc_sent includes 'start' and 'stop'

for j in range(2, n-1): # Going right the columns (obs)
    x = proc_sent[j]  # Obtain j'th word in the (processed) sentence

    for row_no, v in enumerate(all_states): # Going down the rows (states)
        transitions = np.array([ a(u, v) for u in all_states ])
        
        previous_all_scores = (P[j-1, :] * transitions[:, None]).flatten()
        topk = previous_all_scores.argsort()[::-1][:k]
        P[j,row_no] = previous_all_scores[topk] * b(v,x)
        B[j][row_no] = [ all_states[pos // 3] for pos in topk ]

In [14]:
# Termination: j=n-1. Note that proc_sent[n-1] give us the last word in sentence.

# for row_no, v in enumerate(all_states): # Going down the rows (states)
j = n-1
transitions = np.array([ a(u, "STOP") for u in all_states ])
final_topk_scores = (P[j-1] * transitions[:, None]).flatten().argsort()[::-1][:k]

# The following line of code gives us the top 3 parent STATES preceding the STOP state. By top 3, means top 3 best score.
[ all_states[pos // 3] for pos in final_topk_scores ]

# Scratch Code

In [15]:
[ all_states[pos // 3] for pos in final_topk_scores ]

['O', 'O', 'O']

To De Rong: The sentence ends in a fullstop. So the fact that we got `['O', 'O', 'O']` for the top k=3 parent node right before the STOP layer is (kinda) promising. Cos fullstop is supposed to have a state symbol of 'O'. 

In [12]:
P

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.000

In [13]:
B

[[['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-'],
  ['-', '-', '-']],
 [['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-'],
  ['START', '-', '-']],
 [['B-NP', 'O', 'I-ADJP'],
  ['B-NP', 'I-ADJP', 'I-VP'],
  ['B-NP', 'O', 'I-ADJP'],
  ['B-NP', 'O', 'I-ADJP'],
  ['B

In [16]:
all_states

['B-NP',
 'I-NP',
 'B-VP',
 'B-ADVP',
 'B-ADJP',
 'I-ADJP',
 'B-PP',
 'O',
 'B-SBAR',
 'I-VP',
 'I-ADVP',
 'B-PRT',
 'I-PP',
 'B-CONJP',
 'I-CONJP',
 'B-INTJ',
 'I-INTJ',
 'I-SBAR',
 'B-UCP',
 'I-UCP',
 'B-LST']

```
['The', 'quick', 'brown', 'fox', 'jumps', '.']
['B-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP', 'O']
```